# SW정글 week03 컴퓨팅 사고로의 전환
## <DFS!>
220415

In [139]:
# 11725 트리의 부모 찾기
import sys
sys.setrecursionlimit(1000000)
# input = sys.stdin.readline

N = int(input())
adj_list = [[] for _ in range(N+1)]
for _ in range(N-1):
    a, b = map(int, input().split())
    adj_list[a].append(b)
    adj_list[b].append(a)

root = 1
parents = [0] * (N+1)
parents[root] = root

def dfs(node=root):
    for adj_node in adj_list[node]:
        if not parents[adj_node]:
            parents[adj_node] = node
            dfs(adj_node)
dfs()
for node in range(2, N+1):
    print(parents[node])

In [8]:
# 1707 이분 그래프
import sys
sys.setrecursionlimit(10**7)
# input = sys.stdin.readline

for _ in range(int(input())):
    V, E = map(int, input().split())
    adj_list = [[] for _ in range(V+1)]
    for _ in range(E):
        u, v = map(int, input().split())
        adj_list[u].append(v)
        adj_list[v].append(u)

    how_far = [0]*(V+1)
    is_bipartite = True
    def dfs(node):
        global is_bipartite
        if is_bipartite:
            for adj_node in adj_list[node]:
                if not how_far[adj_node]:
                    how_far[adj_node] = how_far[node] + 1
                    dfs(adj_node)

                elif not (how_far[node] - how_far[adj_node])%2: # 사이클이 홀수이면
                    is_bipartite = False
                    return
                    
    for node in range(1, V+1): # 그래프가 한 덩어리가 아닌 경우
        if not is_bipartite:
            break
        if not how_far[node]:
            root = node
            how_far[root] = 1
            dfs(root)
    print('YES') if is_bipartite else print('NO')

YES
NO


In [16]:
# 21606 아침 산책
from collections import deque
import sys
sys.setrecursionlimit(1000000)
# input = sys.stdin.readline

N = int(input())
inner_outer = input().rstrip()
adj_list = [[] for _ in range(N+1)]
for _ in range(N-1):
    u, v = map(int, input().split())
    adj_list[u].append(v)
    adj_list[v].append(u)

def n_comb_2(n):
    return n*(n-1)//2

total_cnt = [0]
visited = [0]*(N+1)
root_queue = deque()

def how_many_path(root):
    now_cnt = []
    def dfs(node=root, group_num=None):
        for adj_node in adj_list[node]:
            if not visited[adj_node]:
                visited[adj_node] = 1
                if inner_outer[adj_node-1] == '0': # 실외
                    if group_num is None:
                        now_cnt.append(0)
                        dfs(adj_node, len(now_cnt)-1)
                    else:
                        dfs(adj_node, group_num)
                else: # 실내
                    if group_num is not None:
                        now_cnt[group_num] += 1
                    else:
                        total_cnt[0] += 1
                    root_queue.append(adj_node)
    dfs()
    
    for cnt in now_cnt:
        total_cnt[0] += cnt + n_comb_2(cnt)

for node in range(1, N+1):
    if inner_outer[node-1] == '1':
        root_queue.append(node)
        visited[node] = 1
        break

while root_queue: # 메모리 줄이려고..
    root = root_queue.popleft()
    how_many_path(root)

print(total_cnt[0]*2)

8


In [43]:
# 14888 연산자 끼워넣기 / 풀고나니 그냥 백트래킹...
def plus(a, b): return a+b
def minus(a, b): return a-b
def mult(a, b): return a*b
def div(a, b):
    if a < 0:
        return -(-a//b)
    return a//b

N = int(input())
A = list(map(int, input().split()))
operator_cnt = list(map(int, input().split()))
operator = (plus, minus, mult, div)

maxi = [-float('inf')]
mini = [float('inf')]
def dfs(n=0, res=A[0]):
    if n == N-1:
        if res > maxi[0]:
            maxi[0] = res
        if res < mini[0]:
            mini[0] = res
        return
    for i, oper in enumerate(operator):
        if operator_cnt[i]:
            operator_cnt[i] -= 1
            dfs(n+1, oper(res, A[n+1]))
            operator_cnt[i] += 1
dfs()
print(maxi[0])
print(mini[0])

54
-24


In [1]:
# 2573 빙산
from collections import deque
from math import ceil
import sys
# input = sys.stdin.readline

N, M = map(int, input().split()) # 행, 열
sea = [list(map(int, input().split())) for _ in range(N)]

year = [0]
iceberg_cnt = [0]
iceberg_info = dict()
visited = [[0]*M for _ in range(N)]

def count_water(coord):
    """coord 위치 주변에 접한 물 수 리턴"""
    row, col = coord
    new_coords = [(row+1, col), (row-1, col), (row, col+1), (row, col-1)]
    w_cnt = 0
    for new_coord in new_coords:
        if 0<=new_coord[0]<N and 0<=new_coord[1]<M and not sea[new_coord[0]][new_coord[1]]:
            w_cnt += 1
    return w_cnt

def initialize_iceberg():
    """iceberg_info 초기화"""
    for row, line in enumerate(sea):
        for col, val in enumerate(line):
            if val:
                iceberg_cnt[0] += 1
                w_cnt = count_water((row, col))
                if w_cnt: # 물과 접한 애들만 등록
                    iceberg_info[(row, col)] = w_cnt

def find_fastest():
    """지금 빙산들 중 가장 빨리 녹을 애의 위치와 걸리는 시간 리턴"""
    mini_year = float('inf')
    fastest_icebergs = []
    for (row, col), water_cnt in iceberg_info.items():
        if water_cnt:
            how_long = ceil(sea[row][col]/water_cnt)
            if how_long < mini_year:
                fastest_icebergs.clear()
                fastest_icebergs.append((row, col))
                mini_year = how_long
            elif how_long == mini_year:
                fastest_icebergs.append((row, col))
    return mini_year, fastest_icebergs

def is_one():
    """빙산이 한 덩어리인지 체크"""
    cnt = 0
    for row, col in iceberg_info:
        one_or_zero = visited[row][col] # visited 재활용하기 위해
        queue = deque([(row, col)])
        visited[row][col] = 1 - one_or_zero
        while queue:
            now_row, now_col = queue.popleft()
            cnt += 1
            for adj_row, adj_col in [(now_row+1, now_col), (now_row-1, now_col), (now_row, now_col+1), (now_row, now_col-1)]:
                if 0<=adj_row<N and 0<=adj_col<M and visited[adj_row][adj_col] == one_or_zero and sea[adj_row][adj_col]>0:
                    queue.append((adj_row, adj_col))
                    visited[adj_row][adj_col] = 1 - one_or_zero
        break

    if cnt == iceberg_cnt[0]:
        return True
    return False

def melt_process(year_change, melted_icebergs):
    """year_change에 맞게 빙산 녹이고 iceberg_info 업데이트"""
    # 녹은 거 업데이트
    for (row, col), water_cnt in iceberg_info.items():
        if water_cnt: # 물과 접한 경우에만
            sea[row][col] -= water_cnt*year_change
    
    # 녹은 빙산 info에서 삭제 및 녹아서 생긴 물에 의한 빙산 info 변화 업데이트
    for row, col in melted_icebergs:
        iceberg_info.pop((row, col))
        for adj_row, adj_col in [(row+1, col), (row-1, col), (row, col+1), (row, col-1)]:
            if 0<=adj_row<N and 0<=adj_col<M and sea[adj_row][adj_col]>0:
                iceberg_info[(adj_row, adj_col)] = iceberg_info.get((adj_row, adj_col), 0) + 1

def process():
    """가장 빨리 녹을 빙산이 녹고, sea와 iceberg_info, year이 업데이트되고
    한 덩어리인지 체크"""
    initialize_iceberg()

    while True:
        year_change, melted_icebergs = find_fastest()
        year[0] += year_change
        
        melt_process(year_change, melted_icebergs)
        iceberg_cnt[0] -= len(melted_icebergs)
        
        if not is_one():
            print(year[0])
            return
        elif iceberg_cnt[0] == 0:
            print(0)
            return

process()

2


In [2]:
# 2617 구슬 찾기
from collections import deque
import sys
# input = sys.stdin.readline

N, M = map(int, input().split())
lighter_list = [[] for _ in range(N+1)]
heavier_list = [[] for _ in range(N+1)]
in_degree, out_degree = [0]*(N+1), [0]*(N+1)
for _ in range(M):
    a, b = map(int, input().split())
    lighter_list[a].append(b)
    in_degree[b] += 1
    heavier_list[b].append(a)
    out_degree[a] += 1

lighter_set, heavier_set = [set() for _ in range(N+1)], [set() for _ in range(N+1)]
lighter_queue, heavier_queue = deque(), deque()
for node in range(1, N+1):
    if in_degree[node] == 0:
        lighter_queue.append(node)
    if out_degree[node] == 0:
        heavier_queue.append(node)

def topology_sort(queue:deque, degree:list, set_list:list, adj_list:list):
    while queue:
        now_node = queue.popleft()
        for adj_node in adj_list[now_node]:
            degree[adj_node] -= 1
            set_list[adj_node].update(set_list[now_node])
            set_list[adj_node].add(now_node)
            if degree[adj_node] == 0:
                queue.append(adj_node)

topology_sort(lighter_queue, in_degree, lighter_set, lighter_list)
topology_sort(heavier_queue, out_degree, heavier_set, heavier_list)

cnt = 0
half = N//2
for node in range(1, N+1):
    if len(lighter_set[node]) > half or len(heavier_set[node]) > half:
        cnt += 1
print(cnt)

2
